In [ ]:
#Downloading dataset spreadsheet
!gdown 
!gdown 

In [ ]:
#Downloading dataset
!gdown 
!unzip 

In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import csv
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
tf.device('/device:GPU:0')

Found GPU at: /device:GPU:0


In [5]:
def mapped_labels(labels, colors, gender, black_white):
    labels_to_lens = {'Colored': 0, 'Normal': 1, 'Transparent': 2}
    lens_colors_types = {"Gray": 0, "Hazel": 1, "Green": 2, "Blue": 3}
    gender_types = {"Male": 0, "Female": 1}
    black_white_types = {"No": 0, "Yes": 1}

    labels = float(labels_to_lens[labels])
    gender = float(gender_types[gender])
    black_white = float(black_white_types[black_white])
    colors = float(lens_colors_types[colors])

    return labels, gender, black_white, colors

In [6]:
def read_csv(train_test, number_of_itens = -1):
    data_labels = []
    imgs = []

    csvreader = None
    with open(f"/content/CogentAnnotation{train_test}.csv", "r") as file:
        csvreader = csv.reader(file)
        csvreader = list(csvreader)

    csvreader.pop(0)
    csvreader = np.array(csvreader)
    np.random.seed(5)
    np.random.shuffle(csvreader)

    for row in csvreader:
        labels, gender, black_white, colors = mapped_labels(row[1], row[3], row[4], row[5])

        img_path = "/content/" + row[0].replace("&", "_") + ".bmp"
        img = img_to_array(load_img(img_path, target_size=(300,300))) / 255.
        imgs.append(img)

        #data_labels.append([labels, colors, black_white, gender, row[6], row[7], row[8], row[9], row[10],row[11]])
        data_labels.append([labels, colors, black_white, gender])


    data_labels = (np.array(data_labels)).astype(np.float32)
    return data_labels.tolist(), imgs

treino_labels, treino_imgs = read_csv("Train")
test_labels, test_imgs = read_csv("Test")

#Converting some part of the test into training
percentage_conversion = 0.7
for i in range (int(len(test_labels) * percentage_conversion)):
  treino_labels.append(test_labels.pop())
  treino_imgs.append(test_imgs.pop())

treino_labels = np.array(treino_labels).astype(np.float32).T
treino_imgs = np.array(treino_imgs).astype(np.float32)
del test_labels
del test_imgs


In [7]:
from keras.backend import sigmoid
from keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects

class Swish(Activation):

    def _init_(self, activation, **kwargs):
        super(Swish, self)._init_(activation, **kwargs)
        self._name_ = 'swish'


def swish(x, beta=1):
    return x * sigmoid(beta * x)


get_custom_objects().update({'swish': Swish(swish)})
tf.keras.utils.get_custom_objects().update({'swish': Swish(swish)})
get_custom_objects().update({'Swish': Swish(swish)})
tf.keras.utils.get_custom_objects().update({'Swish': Swish(swish)})

In [8]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, BatchNormalization, Rescaling, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy

#Modelo base
input = Input(shape=(300,300,3))
rescaling = Rescaling(scale=1./255)(input) #1
base_model = EfficientNetB3(input_shape=(300,300,3), include_top=False)
base_model.trainable = True
model = base_model(rescaling, training=True) #2
batch = BatchNormalization()(model) #3
globalPooling = GlobalAveragePooling2D()(batch) #4
#Camada densa 2
dropout1 = Dropout(0.2)(globalPooling)
dense1 = Dense(512, activation='relu')(dropout1)
#Camada densa 3
dropout2 = Dropout(0.2)(dense1)
dense2 = Dense(256, activation='relu')(dropout2)
#Camada densa 3
dropout3 = Dropout(0.2)(dense2)
final_dense_shared = Dense(128, activation='relu')(dropout3)

# Labels
labels_prediction = Dense(3, activation='softmax', name='labels_prediction')(final_dense_shared)

# Colors
colors_prediction = Dense(4, activation='softmax', name='colors_prediction')(final_dense_shared)

# Black and white
black_and_white_prediction = Dense(1, activation='sigmoid', name='black_and_white_prediction')(final_dense_shared)

# gender
gender_prediction = Dense(1, activation='sigmoid', name='gender_prediction')(final_dense_shared)

new_model = Model(inputs=input, outputs=[labels_prediction, colors_prediction, black_and_white_prediction, gender_prediction])

losses = {
    'labels_prediction': SparseCategoricalCrossentropy(),
    'colors_prediction': SparseCategoricalCrossentropy(),
    'black_and_white_prediction': BinaryCrossentropy(),
    'gender_prediction': BinaryCrossentropy()
}

new_model.compile(optimizer=Adam(learning_rate=0.001), loss = losses, metrics=['accuracy'])


43941136/43941136 [==============================] - 0s 0us/step


In [9]:
history = new_model.fit(
    treino_imgs,
    [treino_labels[0], treino_labels[1], treino_labels[2], treino_labels[3]],
    epochs=20,
    batch_size=16,
    validation_split=0.2
)
print(history)

Epoch 1/20
149/149 [==============================] - 168s 545ms/step - loss: 2.9875 - labels_prediction_loss: 0.7636 - colors_prediction_loss: 1.3887 - black_and_white_prediction_loss: 0.2141 - gender_prediction_loss: 0.6212 - labels_prediction_accuracy: 0.5927 - colors_prediction_accuracy: 0.3083 - black_and_white_prediction_accuracy: 0.9492 - gender_prediction_accuracy: 0.6825 - val_loss: 2.7496 - val_labels_prediction_loss: 0.5985 - val_colors_prediction_loss: 1.3872 - val_black_and_white_prediction_loss: 0.1950 - val_gender_prediction_loss: 0.5689 - val_labels_prediction_accuracy: 0.6600 - val_colors_prediction_accuracy: 0.3132 - val_black_and_white_prediction_accuracy: 0.9497 - val_gender_prediction_accuracy: 0.7069
Epoch 2/20
149/149 [==============================] - 78s 526ms/step - loss: 2.6253 - labels_prediction_loss: 0.6334 - colors_prediction_loss: 1.3114 - black_and_white_prediction_loss: 0.1733 - gender_prediction_loss: 0.5072 - labels_prediction_accuracy: 0.6518 - colo

In [ ]:
try:
  del treino_imgs
  del treino_labels
except:
  pass
try:
  del test_labels
  del test_imgs
except:
  pass

test_labels, test_imgs = read_csv("Test")
#Removing the itens that have gone to the training
for i in range (int(len(test_labels) * percentage_conversion)):
  test_labels.pop()
  test_imgs.pop()

test_labels = np.array(test_labels).astype(np.float32).T
test_imgs = np.array(test_imgs).astype(np.float32)

results = new_model.evaluate(test_imgs, [test_labels[0], test_labels[1], test_labels[2], test_labels[3]])

In [12]:
print("Qtd itens para teste:     ", test_labels.shape[1])
print("Labels:               ", round(results[5]*100, 2), "%")
print("Colors:               ", round(results[6]*100, 2), "%")
print("Black/White:          ", round(results[7]*100, 2), "%")
print("Genero:               ", round(results[8]*100, 2), "%")


Qtd itens para teste:      527
Labels:                89.18 %
Colors:                77.04 %
Black/White:           95.83 %
Genero:                95.07 %
